# Concept Drift - Deployer

In [2]:
import nuclio

In [3]:
from pprint import pprint

In [4]:
%%nuclio cmd -c
python -m pip install scikit-multiflow==0.4.1
python -m pip install v3io_frames

In [5]:
# Define function spec
%nuclio config spec.build.baseImage = "mlrun/ml-models"

%nuclio: setting spec.build.baseImage to 'mlrun/ml-models'


In [6]:
# nuclio: start-code

In [7]:
import skmultiflow.drift_detection
import numpy as np
import pandas as pd
import os
from cloudpickle import dumps, load, dump

from nuclio.triggers import NuclioTrigger
from mlrun import DataItem, import_function, mlconf, MLClientCtx, mount_v3io

# For testing
import random

In [18]:
# Keep until nuclio-jupyter will be updated in the mlrun images
class V3IOStreamTrigger(NuclioTrigger):
    kind = 'v3ioStream'

    def __init__(self, url: str, seekTo: str = 'earliest',
                 partitions: list = [0], pollingIntervalMS: int = 250,
                 readBatchSize: int = 64, maxWorkers: int = 1, 
                 username: str = None, password: str = None):
        self._struct = {'kind': self.kind,
                        'url': url,
                        'attributes': {}}
        if maxWorkers:
            self._struct['maxWorkers'] = maxWorkers
        if seekTo:
            self._struct['attributes']['seekTo'] = seekTo
        if partitions:
            self._struct['attributes']['partitions'] = partitions
        if readBatchSize:
            self._struct['attributes']['readBatchSize'] = readBatchSize
        if pollingIntervalMS:
            self._struct['attributes']['pollingIntervalMs'] = pollingIntervalMS
        user = username if username else os.environ['V3IO_USERNAME']
        self._struct['username'] = user
        access_key = password if password else os.environ['V3IO_ACCESS_KEY']
        self._struct['password'] = access_key

In [10]:
def concept_drift_deployer(context: MLClientCtx, base_dataset:DataItem, 
                           input_stream:str, output_stream:str, output_tsdb:str, tsdb_batch_size:int, callbacks:list, 
                           models:list=['ddm', 'eddm', 'pagehinkley'], models_dest='models',
                           pagehinkley_threshold:float=10, ddm_warning_level:float=2, ddm_out_control_level:float=3,
                           label_col='label', prediction_col='prediction', hub_url:str=mlconf.hub_url, fn_tag:str='master'):
    """Deploy a streaming Concept Drift detector on a labeled stream
       This function is the Deployment step for the Streaming Concept Drift Detector.
       It will load the selected drift detectors and initialize them with the 
       base_dataset's statistics.  Then it will deploy the concept_drift_streaming 
       function and pass the models to it for streaming concept-drift detection on top
       of a labeled stream. 

    :param context: MLRun context
    :param base_dataset: Dataset containing label_col and prediction_col to initialize the detectors
    :param input_stream: labeled stream to track.
                         Should contain label_col and prediction_col
    :param output_stream: Output stream to push the detector's alerts
    :param output_tsdb: Output TSDB table to allow analysis and display
    :param tsdb_batch_size: Batch size of alerts to buffer before pushing to the TSDB
    :param callbacks: Additional rest endpoints to send the alert data to
    :param models: List of the detectors to deploy 
                   Defaults to ['ddm', 'eddm', 'pagehinkley'].
    :param models_dest: Location for saving the detectors 
                        Defaults to 'models' (in relation to artifact_path).
    :param pagehinkley_threshold:  Drift level threshold for PH detector Defaults to 10.
    :param ddm_warning_level:      Warning level alert for DDM detector Defaults to 2.
    :param ddm_out_control_level:  Drift level alert for DDM detector Defaults to 3.
    :param label_col: Label column to be used on base_dataset and input_stream 
                      Defaults to 'label'.
    :param prediction_col: Prediction column to be used on base_dataset and input_stream 
                           Defaults to 'prediction'.
    :param hub_url: hub_url in case the default is not used, concept_drift_streaming will be loaded 
                    by this url 
                    Defaults to mlconf.hub_url.
    :param fn_tag: hub tag to use
                   Defaults to 'master'
    """

    # Set the streaming function
    mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'
    mlconf.hub_url = hub_url
    fn = import_function(url='hub://concept_drift_streaming')
    
    # Load test dataset
    context.logger.info('Loading base dataset')
    base_df = base_dataset.as_df()
    error_stream = np.where(base_df[prediction_col].values==base_df[label_col].values, 0, 1)
    
    # Models
    context.logger.info('Creating models')
    models = [model.strip() for model in os.getenv('models', 'pagehinkley, ddm, eddm').split(',')]
    models = {'eddm': skmultiflow.drift_detection.EDDM(),
              'pagehinkley': skmultiflow.drift_detection.PageHinkley(min_instances=len(error_stream),
                                                                     threshold=pagehinkley_threshold),
              'ddm': skmultiflow.drift_detection.DDM(min_num_instances=len(error_stream),
                                                     warning_level=ddm_warning_level,
                                                     out_control_level=ddm_out_control_level)}
    
    # Run initial dataset
    context.logger.info('Streaming data to models')
    for i in range(len(error_stream)):
        for model_name, model in models.items():
            model.add_element(error_stream[i])
            
    # Save warm models
    context.logger.info('Logging ready models')
    for name, model in models.items():
        data = dumps(model)
        model_file = f'{name}.pkl'
        context.log_model(f'{name}_concept_drift', body=data, labels={'framework': 'skmultiflow', 'workflow': 'concept-drift'},
                          model_file=model_file, model_dir=models_dest, tag='latest')
        fn.set_envs({f'{name}_model_path': os.path.join(context.artifact_path, models_dest, model_file)})
            
    # Deploy streaming concept drift function
    # with the warm models
    context.logger.info('Deploying Concept Drift Streaming function')
    fn.set_envs({'label_col': label_col,
                 'prediction_col': prediction_col, 
                 'drift_stream': output_stream,
                 'tsdb_table': output_tsdb,
                 'pagehinkley_threshold': pagehinkley_threshold,
                 'ddm_warning_level': ddm_warning_level,
                 'ddm_out_control': ddm_out_control_level})    
    fn.add_trigger('labeled_stream', V3IOStreamTrigger(url=input_stream))
    fn.apply(mount_v3io())
    fn.deploy(project=context.project)

In [11]:
# nuclio: end-code

In [12]:
from mlrun import run_local, NewTask

In [13]:
task = NewTask(name='concept_drift_deployer',
        project='network-operations',
        handler=concept_drift_deployer,
        params={'models': ['ddm', 'eddm', 'pagehinkley'],
                'label_col': 'is_error',
                'prediction_col': 'yscore',
                'hub_url': '/User/functions/{name}/function.yaml',
                'output_tsdb': '/bigdata/network-operations/drift_tsdb',
                'input_stream': 'http://v3io-webapi:8081/bigdata/network-operations/inference_stream@cd2',
                'output_stream': '/bigdata/network-operations/drift_stream'},
        inputs={'base_dataset': 'store://network-operations/test_test_set_preds'},
        artifact_path='/User/demo-network-operations/artifacts/')

In [ ]:
run_local(task)

## Save function yaml

In [23]:
from os import path
from mlrun import run_local, NewTask, mlconf, import_function, mount_v3io, code_to_function
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'

In [24]:
# create job function object from notebook code
fn = code_to_function("concept_drift", 
                      kind='job',
                      with_doc=True,
                      embed_code=True)

# add metadata (for templates and reuse)
fn.spec.default_handler = "concept_drift_deployer"
fn.spec.description = "Deploy a streaming Concept Drift detector on a labeled stream"
fn.metadata.categories = ["ml", "serve"]
fn.metadata.labels = {"author": "orz", "framework": "sklearn"}
fn.export("function.yaml")

> 2020-07-27 07:49:39,770 [info] function spec saved to path: function.yaml


In [3]:
fn.apply(mount_v3io())

## Stream testing

In [ ]:
fn.deploy()

In [ ]:
fn.run(task)